In [1]:
import numpy as np
import blottograd.compgraph as cg
import blottograd.nn as nn

import autograd as ag

In [2]:
# let's build a simple example CG
# X -> Linear -> Tanh -> Linear -> Tanh
# X ->  A(X)  -> B(a) - > C(b)  -> D(c)

d_in = 7
d_out = 1
d_hid = 13
w_A = np.random.normal(size=(d_hid, d_in))
w_C = np.random.normal(size=(d_out, d_hid))

start_node = cg.CompGraphNode(nn.Identity(), 'start', None)
node_A = cg.CompGraphNode(nn.Linear(), 'A', w_A)
node_B = cg.CompGraphNode(nn.Tanh(), 'B', None)
node_C = cg.CompGraphNode(nn.Linear(), 'C', w_C)
node_D = cg.CompGraphNode(nn.Tanh(), 'D', None)

# this defines the topology
start_node.children.append(node_A)
node_A.parents.append(start_node)

node_A.children.append(node_B)
node_B.parents.append(node_A)

node_B.children.append(node_C)
node_C.parents.append(node_B)

node_C.children.append(node_D)
node_D.parents.append(node_C)

In [3]:
def test_function(input):
    z = ag.numpy.dot(w_A, input)
    z = ag.numpy.tanh(z)
    z = ag.numpy.dot(w_C, z)
    z = ag.numpy.tanh(z)
    return z

In [4]:
x = np.random.normal(size=d_in)
y_bg = cg.prop_forward(start_node, x)
dydx_bg = cg.prop_backward(start_node, y_bg)

In [5]:
y_ag = test_function(x)
dydx_ag = ag.grad(test_function)(x)

In [6]:
atol = 1e-5
assert np.allclose(y_bg, y_ag, atol=atol)

In [7]:
assert np.allclose(dydx_bg, dydx_ag, atol=atol)